***Universality: Working with one qubit
100 points***

![title](Univ_des1.png)

**Challenge code**

You must complete the get_matrix function below in which you will return the matrix associated with those parameters. Once this is done, a small script will be given to optimize the algorithm looking for the best parameters. In order to do this you must also define the error function that determines how well you are approximating your matrix to the target operator.

***input***

As input to this problem, you are given a 2 x 2 complex matrix that you will try to approximate.

***output***

This code will calculate a list(float) containing the four parameters (, , , and ) using the train_parameters function. Then, the code will output the associated matrix generated from get_matrix.

If your solution matches the correct one within the given tolerance specified in check (in this case it's a 0.2 absolute error tolerance), the output will be "Correct!". Otherwise, you will receive a "Wrong answer" prompt.

Good luck!

***Code***

In [5]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [6]:
np.random.seed(1967)

In [7]:
def get_matrix(params):
    """
    Args:
        - params (array): The four parameters of the model.
    Returns:
        - (matrix): The associated matrix to these parameters.
    """

    alpha, beta, gamma, phi = params

    # Put your code here #
    op = qml.prod(qml.RZ(gamma, wires=0), qml.RX(beta, wires=0), qml.RZ(alpha, wires=0))
    op = qml.s_prod(np.exp(phi*1j), op)
    
    return qml.matrix(op)
def error(U, params):
    """
    This function determines the similarity between your generated matrix and the target unitary.

    Args:
        - U (matrix): Goal matrix that we want to approach.
        - params (array): The four parameters of the model.

    Returns:
        - (float): Error associated with the quality of the solution.
    """

    matrix = get_matrix(params)

    # Put your code here #
    errors = np.sum((np.abs(U-matrix))**2)
    return errors

In [8]:
def train_parameters(U):

    epochs = 1000
    lr = 0.01

    grad = qml.grad(error, argnum=1)
    params = np.random.rand(4) * np.pi

    for epoch in range(epochs):
        params -= lr * grad(U, params)

    return params

In [9]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    matrix = json.loads(test_case_input)
    params = [float(p) for p in train_parameters(matrix)]
    return json.dumps(params)

def check(solution_output: str, expected_output: str) -> None:
    matrix1 = get_matrix(json.loads(solution_output))
    matrix2 = json.loads(expected_output)
    assert not np.allclose(get_matrix(np.random.rand(4)), get_matrix(np.random.rand(4)))
    assert np.allclose(matrix1, matrix2, atol=0.2)

In [10]:
test_cases = [['[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]', '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'], ['[[ 1,  0], [ 0, -1]]', '[[ 1,  0], [ 0, -1]]']]

In [11]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'...
Correct!
Running test case 1 with input '[[ 1,  0], [ 0, -1]]'...
Correct!
